In [1]:
!pip install youtube-dl
!apt-get install -y aria2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aria2 is already the newest version (1.33.1-1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [2]:
!wget https://github.com/curiouswala/EVA-meet-obs-video-sync/blob/main/compute_sound_offset?raw=true -O compute_sound_offset
!wget https://github.com/curiouswala/EVA-meet-obs-video-sync/raw/main/compute-sound-offset.sh -O compute-sound-offset.sh

--2021-01-24 18:46:51--  https://github.com/curiouswala/EVA-meet-obs-video-sync/blob/main/compute_sound_offset?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/curiouswala/EVA-meet-obs-video-sync/raw/main/compute_sound_offset [following]
--2021-01-24 18:46:52--  https://github.com/curiouswala/EVA-meet-obs-video-sync/raw/main/compute_sound_offset
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/curiouswala/EVA-meet-obs-video-sync/main/compute_sound_offset [following]
--2021-01-24 18:46:52--  https://raw.githubusercontent.com/curiouswala/EVA-meet-obs-video-sync/main/compute_sound_offset
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.co

In [3]:
!chmod 755 compute_sound_offset
!chmod 755 compute-sound-offset.sh

In [4]:
import os

obs_link = 'https://www.youtube.com/watch?v=qUcGB38L4_E'
meet_link = 'https://www.youtube.com/watch?v=F4gEcS9hPSI'

In [5]:
print('Downloading OBS version')
os.system(f'youtube-dl -f 22 --external-downloader aria2c {obs_link} -o obs.mp4')
print('Downloading Meet version')

os.system(f'youtube-dl -f 22 --external-downloader aria2c {meet_link} -o gm.mp4')
match_length = 1000 # first n no of seconds which are used for offset calc
# if the starting point of the both videos are far apart, increase it further.
print('Calculating offset')

os.system(f'ffmpeg -ss 0 -t {match_length} -i obs.mp4 obs.wav')
os.system(f'ffmpeg -ss 0 -t {match_length} -i gm.mp4 gm.wav')
p = os.popen('bash compute-sound-offset.sh gm.wav obs.wav 0')
offset_value = p.read().strip()
print('offset value: ',offset_value)

print('Making the final syned video.')
os.system(f'ffmpeg -i obs.mp4 -itsoffset {offset_value} -i gm.mp4 -c copy -map 0:0 -map 1:1 -shortest out.mp4')

Calculating offset
offset value:  6.48275000000000023
Making the final output video.


0

In [6]:
# Downloading the output file from colab is slow
# I just run it locally
from google.colab import files
files.download('out.mp4')

In [7]:
# always clean up before running again.
import os
files = [
    'obs.mp4',
    'gm.mp4',
    'obs.wav',
    'gm.wav',
    'out.mp4'
]

for f in files:
    os.remove(f)